# VROOM testing without ORS
updated with vroom 1.8.0 vroom-express 0.7, supporting Shipments

docs:  https://github.com/VROOM-Project/vroom/blob/v1.8.0/docs/API.md

In [ ]:
import openrouteservice as ors
import requests
import folium
import json
import random
from pprint import pprint
from datetime import datetime
from datetime import date
import time

# DATI posizione consegne
pointsA = json.load(open('../data/mezzolombardo.geojson'))['features'] #4 points
pointsB = json.load(open('../data/mezzocorona.geojson'))['features'] #6 points

example1 = json.load(open('../data/example_1_mezzocorona.json')) #from vroom shipments example

vroom_url = 'http://vroom:3000/optimization/'

center = [46.22007,11.10260]
zoom = 15

point_start = [46.21979,11.10259] #posizione iniziale veicoli
point_end = point_start.copy()

ccA = [p['geometry']['coordinates'] for p in pointsA]
ccB = [p['geometry']['coordinates'] for p in pointsB]

job_locs = ccA + ccB

COLORS = {
    '1': 'blue',
    '2': 'red',
    '3': 'purple',
    '4': 'green'
}

HOURS = [
    [
        hour2sec('08:00'),
        hour2sec('08:05')
    ],
    [
        hour2sec('09:00'),
        hour2sec('09:05')
    ],
    [
        hour2sec('10:00'),
        hour2sec('10:05')
    ],
    [
        hour2sec('11:00'),
        hour2sec('11:05')
    ],
    [
        hour2sec('12:00'),
        hour2sec('12:05')
    ]
]

vehicleProfile = 'driving-car'

Map = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=zoom)


def revcc(cc):
    return list(reversed(cc))

def toKm(m):
    return str(round(m/1000))+'km'

def toMin(sec):
    return str(round(sec/60))+'min'

def sec2hour(sec):
    return str(datetime.fromtimestamp(sec).strftime("%H:%M"))

def hour2sec(hour = '00:00'):
    now = datetime.now()
    s = now.strftime("%d/%m/%Y ") + hour
    return int(time.mktime(datetime.strptime(s, "%d/%m/%Y %H:%M").timetuple()))

def locShift(loc, s):
    return [loc[0] + s, loc[1] + s]
    
def vehicle2Map(v):
    times = v['time_window']
    loc = revcc(v['start'])
    cap = str(v['capacity'][0])
    idv = str(v['id'])
    if times is None:
        times = []
    
    mText = "vehicle %s <br> capacity %s" % (idv, cap)
    
    if len(times) > 0:
        mText += "<br>times from: %s <br> to %s " % (sec2hour(times[0]), sec2hour(times[1]) )
    
    mColor = COLORS[idv]
    mark = folium.Marker(loc, tooltip=folium.Tooltip(mText,permanent=True), icon=folium.Icon(color=mColor, icon='truck', prefix='fa'))
    mark.add_to(Map)

def job2Map(j):
    times = j['time_windows'][0]
    idj = str(j['id'])
    mText = "user %s<br>" % (idj)  #+"quantity %s <br> delivery time %s" % (delivery, toMin(service))
    
    if len(times) > 0:
        mText += "from: %s <br> to: %s " % ( sec2hour(times[0]), sec2hour(times[1]) )
    
    mark = folium.Marker(loc, tooltip=folium.Tooltip(mText, permanent=True), icon=folium.Icon(icon='archive', prefix='fa', color='green'))
    mark.add_to(Map)

vehicles = []

### VEICOLO 1
vid = 1
capacity = 1

timeWindow = [       #orario lavoro del veicolo 1
        hour2sec('08:00'),
        hour2sec('13:00')
    ]

loc = locShift(point_start, 0.004)

vehicle = {
    'id': vid,
    'capacity': [capacity],    # Limite di capacita' del Veicolo    
    'profile': vehicleProfile,
    'start': revcc(loc),       # posizione di partenza veicoli
    'end': revcc(loc),
    'time_window': timeWindow,
#TODO
#     'breaks': [              #se ci sono orari precisi di pause e sanitizzazione del veicolo
#         {
#           "id": 1,
#           #"description": "pausa sanitizzazione", #descrizione pausa
#           "service": 300,   #durata pausa 5 minuti
#           "time_windows": [ #an array of time_window objects describing valid slots for break start
#             [
#               1612881173, #dalle 15:32
#               1612882173  #alle 15:49
#             ],
#             [
#               1612884173, #dalle 16:22
#               1612885173  #alle 16:39
#             ]
#           ]
#         }
#       ]
}
vehicles.append(vehicle)

### VEICOLO 2
vid = 2
capacity = 1

timeWindow = [                  # orario lavoro del veicolo 2
        hour2sec('14:00'),
        hour2sec('19:00')
    ]

loc = locShift(point_start, 0.003)

vehicle = {
    'id': vid,
    'capacity': [capacity],     # Limite di capacita' del Veicolo    
    'profile': vehicleProfile,
    'start': revcc(loc),        # posizione di partenza veicoli
    'end': revcc(loc),
    'time_window': timeWindow
}
vehicles.append(vehicle)

#DEFINTION OF JOBS

jobs = []
for jid, cc in enumerate(job_locs):
    loc = revcc(cc)
    
    #quantita da consegnare
    delivery = 1 #random.randrange(1, 3)
    
    #tempo di della singola consegna/ritiro(in secondi)
    service = random.randrange(60, 300) #da 2 a 5 minuti
    
    timeWindow = random.choice(HOURS)
    
    # shifting casuale del tempo per ogni job
    #s = random.randrange(-3000,3000)
    #timeWindow = [timeWindow[0]+s, timeWindow[1]+s]
    
    job = {
        'id': jid,
        'location': cc,
        'delivery': [delivery], # Quantita' da consegnare
        #'service': service,      # tempo di consegna o aggiunta del tempo DI PULIZIA mezzo
        'time_windows': [
            timeWindow
        ]
    }
    jobs.append(job)


request = {
    'vehicles': vehicles,
    'jobs': jobs
}

request = example1

# RENDERIZZA PROBLEMA RICHIESTO IN MAP

In [12]:
import openrouteservice as ors
import requests
import folium
import json
import random
from pprint import pprint
from datetime import datetime
from datetime import date
import time

# DATI posizione consegne
pointsA = json.load(open('../data/mezzolombardo.geojson'))['features'] #4 points
pointsB = json.load(open('../data/mezzocorona.geojson'))['features'] #6 points

example1 = json.load(open('../data/example_1_mezzocorona.json')) #from vroom shipments example

vroom_url = 'http://vroom:3000/optimization/'

center = [46.22007,11.10260]
zoom = 15

point_start = [46.21979,11.10259] #posizione iniziale veicoli
point_end = point_start.copy()

ccA = [p['geometry']['coordinates'] for p in pointsA]
ccB = [p['geometry']['coordinates'] for p in pointsB]

job_locs = ccA + ccB

COLORS = {
    '1': 'blue',
    '2': 'red',
    '3': 'purple',
    '4': 'green'
}

HOURS = [
    [
        hour2sec('08:00'),
        hour2sec('08:05')
    ],
    [
        hour2sec('09:00'),
        hour2sec('09:05')
    ],
    [
        hour2sec('10:00'),
        hour2sec('10:05')
    ],
    [
        hour2sec('11:00'),
        hour2sec('11:05')
    ],
    [
        hour2sec('12:00'),
        hour2sec('12:05')
    ]
]

vehicleProfile = 'driving-car'

Map = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=zoom)


def revcc(cc):
    return list(reversed(cc))

def toKm(m):
    return str(round(m/1000))+'km'

def toMin(sec):
    return str(round(sec/60))+'min'

def sec2hour(sec):
    return str(datetime.fromtimestamp(sec).strftime("%H:%M"))

def hour2sec(hour = '00:00'):
    now = datetime.now()
    s = now.strftime("%d/%m/%Y ") + hour
    return int(time.mktime(datetime.strptime(s, "%d/%m/%Y %H:%M").timetuple()))

def locShift(loc, s):
    return [loc[0] + s, loc[1] + s]
    
def vehicle2Map(v):
    times = v['time_window']
    loc = revcc(v['start'])
    cap = str(v['capacity'][0])
    idv = str(v['id'])
    if times is None:
        times = []
    
    mText = "vehicle %s <br> capacity %s" % (idv, cap)
    
    if len(times) > 0:
        mText += "<br>times from: %s <br> to %s " % (sec2hour(times[0]), sec2hour(times[1]) )
    
    mColor = COLORS[idv]
    mark = folium.Marker(loc, tooltip=folium.Tooltip(mText,permanent=True), icon=folium.Icon(color=mColor, icon='truck', prefix='fa'))
    mark.add_to(Map)

def job2Map(j):
    times = j['time_windows'][0]
    idj = str(j['id'])
    mText = "user %s<br>" % (idj)  #+"quantity %s <br> delivery time %s" % (delivery, toMin(service))
    
    if len(times) > 0:
        mText += "from: %s <br> to: %s " % ( sec2hour(times[0]), sec2hour(times[1]) )
    
    mark = folium.Marker(loc, tooltip=folium.Tooltip(mText, permanent=True), icon=folium.Icon(icon='archive', prefix='fa', color='green'))
    mark.add_to(Map)

vehicles = []

### VEICOLO 1
vid = 1
capacity = 1

timeWindow = [       #orario lavoro del veicolo 1
        hour2sec('08:00'),
        hour2sec('13:00')
    ]

loc = locShift(point_start, 0.004)

vehicle = {
    'id': vid,
    'capacity': [capacity],    # Limite di capacita' del Veicolo    
    'profile': vehicleProfile,
    'start': revcc(loc),       # posizione di partenza veicoli
    'end': revcc(loc),
    'time_window': timeWindow,
#TODO
#     'breaks': [              #se ci sono orari precisi di pause e sanitizzazione del veicolo
#         {
#           "id": 1,
#           #"description": "pausa sanitizzazione", #descrizione pausa
#           "service": 300,   #durata pausa 5 minuti
#           "time_windows": [ #an array of time_window objects describing valid slots for break start
#             [
#               1612881173, #dalle 15:32
#               1612882173  #alle 15:49
#             ],
#             [
#               1612884173, #dalle 16:22
#               1612885173  #alle 16:39
#             ]
#           ]
#         }
#       ]
}
vehicles.append(vehicle)

### VEICOLO 2
vid = 2
capacity = 1

timeWindow = [                  # orario lavoro del veicolo 2
        hour2sec('14:00'),
        hour2sec('19:00')
    ]

loc = locShift(point_start, 0.003)

vehicle = {
    'id': vid,
    'capacity': [capacity],     # Limite di capacita' del Veicolo    
    'profile': vehicleProfile,
    'start': revcc(loc),        # posizione di partenza veicoli
    'end': revcc(loc),
    'time_window': timeWindow
}
vehicles.append(vehicle)

#DEFINTION OF JOBS

jobs = []
for jid, cc in enumerate(job_locs):
    loc = revcc(cc)
    
    #quantita da consegnare
    delivery = 1 #random.randrange(1, 3)
    
    #tempo di della singola consegna/ritiro(in secondi)
    service = random.randrange(60, 300) #da 2 a 5 minuti
    
    timeWindow = random.choice(HOURS)
    
    # shifting casuale del tempo per ogni job
    #s = random.randrange(-3000,3000)
    #timeWindow = [timeWindow[0]+s, timeWindow[1]+s]
    
    job = {
        'id': jid,
        'location': cc,
        'delivery': [delivery], # Quantita' da consegnare
        #'service': service,      # tempo di consegna o aggiunta del tempo DI PULIZIA mezzo
        'time_windows': [
            timeWindow
        ]
    }
    jobs.append(job)


request = {
    'vehicles': vehicles,
    'jobs': jobs
}

request = example1

# RENDERIZZA PROBLEMA RICHIESTO IN MAP
for vehicle in request.vehicles:
    vehicle2Map(vehicle)

for job in request.jobs:
    job2Map(job)
Map

AttributeError: 'dict' object has no attribute 'vehicles'

In [9]:
#request

In [5]:
request['geometry'] = True

resp = requests.post(vroom_url, json = request)

solution = resp.json()

#solution['summary']
solution

NameError: name 'request' is not defined

In [11]:
if len(solution['routes']) > 0:
    for route in solution['routes']:

        rtext = 'distance: '+toKm(route['distance'])+' delivered quantity: '+str(route['delivery'][0])+', time: '+toMin(route['duration']+route['service'])
        rcolor = COLORS[str(route['vehicle'])]
        print('Vehicle'+str(route['vehicle']), rcolor, rtext)

        folium.PolyLine(
            locations = [revcc(coords) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']],
            color = rcolor,
            opacity = 0.8,
            weight = 5
        ).add_to(Map)

    print("\nTotal distance: "+toKm(solution['summary']['distance']), 'Not Delivered: '+str(len(solution['unassigned'])))

Map

Vehicle2 red distance: 3km delivered quantity: 1, time: 4min

Total distance: 3km Not Delivered: 9


In [6]:
solution

NameError: name 'solution' is not defined